In [ ]:
import face_rec
import numpy as np
import pandas as pd
import cv2
import time
from datetime import datetime

import redis
from insightface.app import FaceAnalysis
from sklearn.metrics import pairwise

In [ ]:
face_rec.r.ping()

In [ ]:
name='academy:register'
retrive_dict=face_rec.r.hgetall(name)
retrive_series=pd.Series(retrive_dict)
retrive_series=retrive_series.apply(lambda x: np.frombuffer(x,dtype=np.float32))
index=retrive_series.index
index=list(map(lambda x: x.decode(), index))
retrive_series.index=index
retrieve_df=retrive_series.to_frame().reset_index()
retrieve_df.columns=['name_role','Facial_features']
retrieve_df[['Name','Role']]=retrieve_df['name_role'].apply(lambda x: x.split('@')).apply(pd.Series)
retrieve_df



In [ ]:
retrieve_df['Facial_features'].apply(lambda x: len(x) if isinstance(x, np.ndarray) else None)


In [ ]:
retrieve_df['Facial_features'].apply(lambda x: len(x) if isinstance(x, np.ndarray) else None)

In [6]:
waitTime=30
setTime=time.time()
realtimepred=face_rec.RealTimePred()


In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame=cap.read()
    
    if ret == False:
        break
    
    pred_frame=realtimepred.face_prediction(frame, retrieve_df, 'Facial_features', ['Name','Role'],thresh=0.5)
    

    
   
    cv2.imshow('frame',frame)
    cv2.imshow('prediction',pred_frame)
    timenow=time.time()
    difftime=timenow-setTime
    if difftime>=waitTime:
        realtimepred.saveLogs_redis()
        setTime=time.time()
        
        print('Save Data to redis database')
        
    
    if cv2.waitKey(1)==27:
        break
cv2.destroyAllWindows()
cap.release()
    

In [ ]:

# Check if the key exists
key = "attendence:logs"

if face_rec.r.exists(key):
    # Retrieve all the data from the key
    data = face_rec.r.lrange(key, 0, -1)  # Retrieve all elements from the list
    print(f"Contents of the key '{key}':")
    for item in data:
        print(item)
else:
    print(f"The key '{key}' does not exist in Redis.")